# Classification and Logistic Regression

Lets now talk about the classification problem. This is just like the regression problem, except that the vlaues y are now want to predict take on only a small number of discrete values. For now, we will focus on the **binary classification** problem in which y can take on two values, 0 and 1. (Most of what this notebook say here will also generalize to the multiple-class case). For instance, if we are trying to build a spam classifier for email, then $x^{(i)}$ may be some features of a piece of email, and $y$ may be 1 if it is a piece of spam mail, and 0 otherwise. 0 is called **negative class**, and 1 the **positive class**, they are sometimes also denoted by the symbols "$-$" and "$+$". Given $x^{(i)}$, the corresponding $y^{(i)}$ is also called the **label** for the training example.

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
cd /content/drive/MyDrive/files

/content/drive/MyDrive/files


In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12.0, 9.0)

In [42]:
df = pd.read_csv("iris.csv")
df = df.sample(frac=1).reset_index(drop=True)
df[:5]

,sepal_length,sepal_width,petal_length,petal_width,class
0,6.2,2.8,4.8,1.8,Iris-virginica
1,5.0,2.3,3.3,1.0,Iris-versicolor
2,6.5,3.0,5.5,1.8,Iris-virginica
3,6.2,3.4,5.4,2.3,Iris-virginica
4,6.3,2.5,5.0,1.9,Iris-virginica


As we are working on the logistic regression and where the target variable $\in \{0, 1\}$, its better to convert the class variable to the

In [43]:
df['class'].unique()
df['class'] = df['class'].apply(lambda x: 1 if x == 'Iris-virginica' else 0 )
df[:5]

,sepal_length,sepal_width,petal_length,petal_width,class
0,6.2,2.8,4.8,1.8,1
1,5.0,2.3,3.3,1.0,0
2,6.5,3.0,5.5,1.8,1
3,6.2,3.4,5.4,2.3,1
4,6.3,2.5,5.0,1.9,1


Here x's are four-dimensional vector in $\mathbb{R}^4$. For instance, $x_1^{(i)}$ is the `sepal_length`, $x_2^{(i)}$ is the `sepal_width`, $x_3^{(i)}$ is the `petal_length`, and $x_4^{(i)}$ is the `petal_width` of the i-th house in the training set.<br>
Hypothesis for the $X$ & $\theta$(parameters) can be written as

$$\begin{align*}
h_\theta(x) &= \theta_0 + \theta_1x_1 + \theta_2x_2 + \theta_3x_3 + \theta_4x_4 \\
&= \sum_{i=0}^{d}\theta_ix_i \text{ where $x_0$ is intercept term}\\
&= \theta^Tx
\end{align*}$$


In [35]:
# to replicate the values across, lets set the random seed
np.random.seed(seed=123456)
X = df[['sepal_length','sepal_width', 'petal_length', 'petal_width']].to_numpy()
y = df[['class']].to_numpy()
#We then need to add a feature of “1” concatenating it with the dataset we already have and also add q to the vector m.
X = np.concatenate([X , np.ones((X.shape[0],1))], axis = 1)

In [36]:
# contains the theta random theta values from θ1, θ2, θ3, θ4
θ_1_4 = np.random.rand(4,1)

# creating the θ0(theta zero or intecept)
θ0 = np.random.rand(1)

# complete hθ
hθ = np.concatenate([θ_1_4,θ0.reshape(1,-1)],axis = 0)
print(hθ.shape)
print(hθ)


(5, 1)
[[0.12696983]
 [0.96671784]
 [0.26047601]
 [0.89723652]
 [0.37674972]]


#2.1 Logistic regression

We could approach the classification problem ignoring the fact that $y$ is discrete-valued, and use old linear regression algorithm to try to predict $y$ given $x$. However, it is easy to construct examlpes where this method performs very poorly. Intuitively, it also deosn't make sense for $h_\theta(x)$ to take values larger than 1 or smaller than 0 when we know that $ y \in \{0, 1\}$.

To fix this, let's change the form for our hypothesis $h_\theta(x)$. We will choose
$$h_\theta(x) = g(\theta^Tx) = \frac{1}{1 + e^{-\theta^Tx}},$$
where
$$g(z) = \frac{1}{1+e^{-z}}$$
is called the **logistic function** or the **sigmoid function**. Here is a plot showing $g(z)$:
<figure>
  <img src="TraditionalML/images/sigmoid_activation_function.png" width="40%"/>
  <figcaption>Ref: https://cs229.stanford.edu/notes2021fall/cs229-notes1.pdf</figcaption>
</figure>
 <br>

Notice that $g(z)$ tends towards 1 as $z \to \infty$, and $g(z)$ tends towards $0$ as $z \to -\infty$. Moreover, $g(z)$, and hence also $h(x)$, is always bounded between $0$ and $1$. As before, we are keeping the convention of letting $x_0 = 1$, so that $ \theta^Tx = \theta_0 + \sum_{j=1}^d\theta_jx_j$.<br>



In [37]:
# lets define sigmoid function
def sigmoid(Z):
    return 1/(1 + np.exp(-Z))

For now, let's take the choice of $g$ as given. Other functions that smoothly increase from $0$ to $1$ can also be used, but for a couple of reasons that we'll see later( when we talk about GLMs, and when we talk about generative learning algorithms), the choice of the logistic function is a fairly natural one. Before moving on, here's a useful property of the derivative of the signmoid fucntion, which we write as $g'$:<br>

$$\begin{align*}g'(z) &= \frac{d}{dz}\frac{1}{1 + e^{-z}} \\
&= \frac{1}{(1 + e^{-z})^2}(e^{-z}) \\
&= \frac{1}{(1 + e^{-z})}.\left(1 - \frac{1}{(1 + e^{-z})}\right)\\
&=g(z)(1-g(z))
\end{align*}$$

 So, given the logistic regression model, how do we fit $\theta$ for it? Following how we saw least squares regression could be derived as the maximum like-lihood estimator under a set of assumptions, let's endow our classficitiona model with a set of probabilistc assumptions, lets endow our classifiction model with a set of probabilistic assumptions, and then fit the parameters via maximum likelihood. <br>

 Let us assume that
 $$\begin{align*}
 P(y = 1 | x; \theta) &= h_\theta(x) \\
 P(y = 0 | x; \theta) &= 1 - h_\theta(x)
 \end{align*}$$

 <br>Note that this can be written more compactly as<br>
 $$\begin{align*}
 p(y | x;\theta) = (h_\theta(x))^y (1 - h_\theta(x))^{1-y}
 \end{align*}$$
 <br>
 Assuming that the $n$ traininig examples were generated independently, we can then write down the likelihood of the parameters as
 $$\begin{align*}
 L(\theta) &= p(\vec{y}| X;\theta) \\
 &= \prod_{i=1}^np(y^{(i)} | x^{(i)}; \theta) \\
 &= \prod_{i=1}^np(h_\theta(x^{(i)}))^{y^{(i)}}(1 - h_\theta(x^{(i)}))^{1 - y^{(i)}}
\end{align*}$$

As before, it will be easier to maximize the log likelihood:
$$\begin{align*}
l(\theta) &= \text{log} L(\theta) \\
&= \sum_{i=n}^ny^{(i)}\text{log}h(x^{(i)}) + (1 - y^{(i)})\text{log}(1 - h(x^{(i)}))
\end{align*}$$



In [65]:
# we have defined the cost fuction lθ, which is modelled after the mean square error
def max_log_likelihood(X, y, hθ):
    hx = X @ hθ
    gx = sigmoid(hx)
    print(y.shape, y.T.shape, 'y')
    print(gx.shape, gx.T.shape, 'X')
    lθ = np.sum(np.dot(y.T,np.log(gx)) + np.dot((1-y).T,np.log(1-gx)))
    print(lθ, "max_log_likelihood")
    return lθ

How do we maximize the likelihood? Similar to our derivation in the case of linear regression, we can use gradient ascent. Written in vectorial notation, our update will therefore be given by $ \theta := \theta + \alpha\Delta_\theta l(\theta)$. (Note the positive rather than negative sign in the update formula, since we'are maximizing, rather than minimizing, a function now.) Let's start by working with just one training examle $(x,y)$, and take derivatives to derive the stochastic gradient ascent rule:

$$
\begin{align*}
\frac{\partial}{\partial \theta_j} l(\theta) &= \left( y \frac{1}{g(\theta^Tx)} - (1-y)\frac{1}{1-g(\theta^Tx)}  \right)\frac{\partial}{\theta_j}g(\theta^Tx)\\
&= \left( y \frac{1}{g(\theta^Tx)} - (1-y)\frac{1}{1-g(\theta^Tx)}  \right) g(\theta^Tx)(1 - g(\theta^Tx))\frac{\partial}{\partial \theta_j}\theta^Tx \\
&= (y(1 - g(\theta^Tx)) - (1-y)g(\theta^Tx))x_j \\
&=(y -h\theta(x))x_j
\end{align*}$$
<br>
Above, we used the fact that $g'(z) = g(z)(1-g(z))$. This therefore gives us the stochastic gradient ascent rule:
$$ \theta_j := \theta_j + \alpha(y^{(i)} - h_\theta(x^{(i)}))x_j^{(i)}$$
<br>
If we compare this to the LMS update rule, we that it looks identical; but this $\textit{not}$ the same algorithm, becuase $h_\theta(x^{(i)})$ is now defined as a non-linear function of $\theta^Tx^{(i)}$. Nonetheless, its a little suprising that we end up with the same update rule for a rather different algorithm and learning problem.

$\cdots \vdots \ddots$

In [70]:
def partial_derivative(X_batch, y_batch, hθ):
    y_pred = sigmoid(X_batch @ hθ)

    df_dhθ = (X_batch.T @ (y_batch - y_pred))
    df_dhθ = df_dhθ.reshape(len(df_dhθ), -1)
    print(df_dhθ)
    return df_dhθ

In [71]:
def training_batch(X, y, batch_size, lr, epochs, hθ):

    for epoch in range(epochs):

        # shuffle X and y using same permutation
        indices = np.arange(X.shape[0])
        np.random.shuffle(indices)

        X = X[indices]
        y = y[indices]

        # store cumulative derivative
        cumulative_derivative = np.zeros((X.shape[1], 1))

        for batch in range(len(X)//batch_size):
            start = batch*batch_size
            stop = (batch*batch_size) + batch_size

            X_batch = X[start:stop]
            y_batch = y[start:stop]
            parti = partial_derivative(X_batch, y_batch, hθ)
            cumulative_derivative = cumulative_derivative + parti

            #updating rule
            hθ = hθ - (lr * cumulative_derivative)

        #print(f"m_stat: {m_stat}")
        print(f"epoch: {epoch} ----> MSE: {max_log_likelihood(X, y, hθ)}")
    return hθ


In [73]:
batch_size = 25
lr = 0.001
epochs = 1

hθ = training_batch(X,y, batch_size,lr,epochs, hθ)
hθ

[[-87.3]
 [-50.5]
 [-45.1]
 [-12. ]
 [-16. ]]
[[-81.9]
 [-48.3]
 [-39.6]
 [-11.2]
 [-15. ]]
[[-87.5]
 [-51. ]
 [-41.7]
 [-11. ]
 [-16. ]]
[[-92.5]
 [-50.9]
 [-53.1]
 [-15.1]
 [-17. ]]
[[-88. ]
 [-48.3]
 [-46.9]
 [-12.6]
 [-16. ]]
[[-109.9]
 [ -60.4]
 [ -59.8]
 [ -16.6]
 [ -20. ]]
(150, 1) (1, 150) y
(150, 1) (1, 150) X
nan max_log_likelihood
epoch: 0 ----> MSE: nan


<ipython-input-65-8bf4504f55f7>:7: RuntimeWarning: divide by zero encountered in log
  lθ = np.sum(np.dot(y.T,np.log(gx)) + np.dot((1-y).T,np.log(1-gx)))


array([[19.44307359],
       [11.82165636],
       [10.55295817],
       [ 3.73735845],
       [ 3.87760979]])

In [56]:
np.dot(np.array([[1], [2]]), np.array([[1], [2]]).reshape(1, 2))

array([[1, 2],
       [2, 4]])

In [57]:
np.array([[1], [2]]).reshape(1, 2)

array([[1, 2]])